import modules

In [26]:
import pandas as pd
import os
current_directory = os.getcwd()
print("현재 디렉토리:", current_directory)

현재 디렉토리: c:\apps\Project1\mid_code


load data

In [27]:
cny = pd.read_csv('../dataset/CNY_KRW.csv').drop(['고가','저가','거래량'], axis=1).rename(
    columns={'날짜':'date',
             '종가':'close',
             '시가':'open',
             '변동 %':'close_change'})

In [28]:
cny.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          66 non-null     object 
 1   close         66 non-null     float64
 2   open          66 non-null     float64
 3   close_change  66 non-null     object 
dtypes: float64(2), object(2)
memory usage: 2.2+ KB


#### 환율 변동값 전처리
* 전일 변동율 = (전날종가-전날시가)/전날시가 yesterday_change <br>
* 야간 변동율 = (당일시가-전날종가)/전날종가 night_change <br>
* 데이터 기간 : 2023/07/31 ~ 2023/10/20

In [29]:
# 파생변수 생성 (전일 변동율 = yesterday_change)
cny['cny_yesterday_change'] = round((cny['close']-cny['open'])/cny['open']*100, 2)
cny['cny_yesterday_change'] = cny['cny_yesterday_change'].shift(1)

# 파생변수 생성(야간 변동율 = night_change)
cny['close_shift'] = cny['close'].shift(1)
cny['cny_night_change'] = round((cny['open']-cny['close_shift'])/cny['close_shift']*100,2)

cny.drop('close_shift',axis=1,inplace=True)

In [30]:
# 종가 기준 변동율 전처리 및 데이터 형태 변환( 단위 : % )
cny['close_change'] = cny['close_change'].str.replace('%','')
cny['close_change'] = cny['close_change'].astype(float)

In [31]:
# 데이터 타입 변경
cny['date'] = pd.to_datetime(cny['date'])

In [32]:
# 데이터 기간 : 2023/07/31 ~ 2023/10/20
cny = cny.loc[cny['date']<='2023-10-20',].loc[cny['date']>='2023-07-31',]
cny = cny[['date','cny_yesterday_change','cny_night_change']]

In [33]:
# 날짜순으로 오름차순 정렬
cny.sort_values(by='date', inplace=True)
cny.reset_index(drop=True, inplace=True)

In [34]:
cny

,date,cny_yesterday_change,cny_night_change
0,2023-07-31,0.35,-0.82
1,2023-08-01,0.53,-0.91
2,2023-08-02,0.30,-0.88
3,2023-08-03,0.25,-0.64
4,2023-08-04,-0.09,-0.18
5,2023-08-07,0.30,-0.27
6,2023-08-08,-0.22,-0.13
7,2023-08-09,0.15,0.04
8,2023-08-10,0.62,-0.88
9,2023-08-11,0.22,-0.87


In [35]:
cny.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  60 non-null     datetime64[ns]
 1   cny_yesterday_change  60 non-null     float64       
 2   cny_night_change      60 non-null     float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 1.5 KB


csv 파일로 새로 저장

In [36]:
cny.to_csv("../dataset/clean_feature/cny_krw_final.csv")